In [1]:
burrowactivate()
import ERA5_Analysis as ERA
using PyCall, Dictionaries, DataFrames, Shapefile
@pyimport matplotlib.pyplot as plt
@pyimport matplotlib.patches as mpatches
@pyimport cartopy.crs as ccrs

  Activating environment at `~/Desktop/ERA5_Analysis/Project.toml`
┌ Info: Precompiling ERA5_Analysis [a4f775dc-9ac1-4079-aa93-d12d58c0fe66]
└ @ Base loading.jl:1342
┌ Warning: Package ERA5_Analysis does not have StaticArrays in its dependencies:
│ - If you have ERA5_Analysis checked out for development and have
│   added StaticArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ERA5_Analysis
└ Loading StaticArrays into ERA5_Analysis from project dependency, future warnings for ERA5_Analysis are suppressed.


In [2]:

#use for allowed ids
include(joinpath(ERA.NRCSDATA, "wanted_stations.jl"))

huc_paths = joinpath.(ERA.ERA5DATA, "basin_extractions/data/HUC_Shapes/WBDHU".*string.([6,8]).*".shp")
shapes = Dictionary([6,8],DataFrame.(Shapefile.Table.(huc_paths)))

ax = plt.subplot(1,1,1,projection = ccrs.AlbersEqualArea(central_longitude = -150,
            standard_parallels = (57,69)))

basincolors = Dictionary(basin_names,["red","green","yellow","orange","cyan"])

for (basin_ids, basin_name) in zip(allowed_ids, basin_names)
    for huc in basin_ids
        shapefile = shapes[length(huc)]
        basin_idx = findfirst( ==(huc), shapefile[!, "huc$(length(huc))"])
        #and use that to grab the polygon
        my_polygon = shapefile[basin_idx, :geometry]
        poly_verts = hcat(getproperty.(my_polygon.points, :x), getproperty.(my_polygon.points, :y))
        poly=mpatches.Polygon(poly_verts, closed=true, alpha=0.7, facecolor = basincolors[basin_name],
        transform = ccrs.PlateCarree())
        ax.add_patch(poly)
    end
end
ax.set_extent(ERA.ak_bounds, crs=ccrs.PlateCarree())
ax.gridlines()
ax.coastlines()
plt.savefig("basin_vis.png")